In [ ]:
# Fonte de dados - Vila Velha - ES

from bs4 import BeautifulSoup
import requests

url = 'https://www.vilavelha.es.leg.br/institucional/diario-oficial/2020-1'
r  = requests.get(url)

data = r.text
soup = BeautifulSoup(data)
lista_links_vv = []
lista_datas_vv = []

# Obtendo links dos diarios atraves do beautiful soup

for link in soup.find_all('h2'):
    # print(link.find('a').get('href'))
    lista_links_vv.append(link.find('a').get('href'))
    #lista_links_vv.append(str(url)+'/'+str(link.get('href')))

In [ ]:
import wget
import os

# Obtem os pdfs via wget
# Aqui estou obtendo apenas de um dia

wget.download("https://www.vilavelha.es.leg.br/institucional/diario-oficial/diario-no-3611-dia-09-12-2020/at_download/file")

for arquivo in lista_links_vv:
#    print(arquivo)
    arquivo = arquivo.replace('view','at_download/file')
#    print(arquivo)
    wget.download(arquivo)


In [ ]:
# Movendo para a pasta temp

arr = os.listdir('c:\\temp\\vilavelha_leg')

for arq in arr:
    print(arq)


In [ ]:
################ ERRO DE PERMISSAO

from bs4 import BeautifulSoup
import requests
import wget
import os


url3 = 'https://www.vilavelha.es.gov.br/diariooficial/'
r3  = requests.get(url3)

data3 = r3.text
soup3 = BeautifulSoup(data3)
lista_links3 = []
lista_datas3 = []

print('##### obtendo links')

for link in soup3.find_all('a'):
    if str(link.get('href')).count('lbDownloadPDF') > 0: 
        print(url3+link.get('href'))
        #lista_links3.append(url3+link.get('href'))
        #wget.download(link.get('href'))
        wget.download(url3+'Diario_Oficial_VilaVelha_09-12-2020_1083_1.pdf')
        
        # Diario_Oficial_VilaVelha_09-12-2020_1083_1.pdf
        #javascript:__doPostBack('ctl00$cpConteudo$gvUltimosDocumentos$ctl02$lbDownloadPDF','')

In [ ]:
# PORTARIAS
# 5660 01/10/2020 NOMEIA ERICK DE OLIVEIRA CARDOSO PROCURADOR GERAL


from PyPDF2 import PdfFileReader
import re
import os
import csv
import sys

for filename in os.listdir("C:\\temp\\vilavelha_leg\\"):
    if filename.endswith(".pdf"): 
        input = PdfFileReader("C:\\temp\\vilavelha_leg\\"+filename)
        NumPages = input.getNumPages()

        String = "NOMEIA"
        StringFim = "\n"

        for i in range(0, NumPages):
           # print("pag"+str(i))
            PageObj = input.getPage(i)
            Text = PageObj.extractText() 
            try:
                
                #  dd\mm\YYYY 
                
                expRegular = "NOMEIA(.*?)\n(.*?)\n(.*?)\n"
                
                x1 = re.findall(expRegular, Text, re.DOTALL) 
                #print(len(x1))
              
                
                if len(x1) > 0:
                    #print(x1)
                    # Obtem o cargo

                    # grava arquivo csv
                    with open('C:\\temp\\vilavelha_leg\\'+filename+'_FINAL.csv', 'a', newline='') as csvfile:
                        for linha in range(len(x1)):
                            writer = csv.writer(csvfile, delimiter=';')
                            lista = [0,x1[linha],x1[linha],filename,"Vila Velha","ES"]
                            writer.writerow(lista)
                    
            except:
                print("excecao " + filename + sys.exc_info()[0])
                continue


In [ ]:
### Vila Velha
################# Definitivo com a api correta

import fitz 
import re
import os
import csv

for filename in os.listdir("C:\\temp\\vilavelha_leg\\"):
    if filename.endswith(".pdf"): 
        input = doc = fitz.open("C:\\Temp\\vilavelha_leg\\"+filename)

        String = "NOMEIA"
        StringFim = "\n"
        
        text_list=[] 
        for i in range(input.pageCount):  
            pg=input.loadPage(i) 
            Text=pg.getText() 
            #text_list.append(Text) 
            ResSearch1 = re.search(String, Text)
            ResSearch2 = re.search(StringFim, Text)
            if Text.partition(String)[2].partition(StringFim)[0] is not "":
                aux = Text.partition(String)[2].partition(StringFim)[0]
                try:
                    # Obtem o nome da pessoa, após reduzir para o trecho de nomeação
                    expRegular = "NOMEIA(.*?)\n(.*?)\n(.*?)\n"
                    
                    x1 = re.search(expRegular, aux, re.DOTALL) 
                
                   # print(filename)
                
                    # Obtem o cargo
                    expRegularCargo1 = x1.group()+"\s{0,}para\s{0,}(.*)\s{0,}cargo\s{0,}(.*)\s{0,}(.*)\s{0,}(.*)\s{0,}\."
                    cargo1=re.search(expRegularCargo1,aux)
                    cargoAux = ""
                                    
                    for i in range(2,len(cargo1.groups())):
                        cargoAux = cargoAux + cargo1.group(i)
                        print("G" + str(i) + " "+ cargo1.group(i))
                    
                    # grava arquivo csv
                    with open('C:\\temp\\es_gov\\'+filename+'_FINAL.csv', 'a', newline='') as csvfile:
                        writer = csv.writer(csvfile, delimiter=';')
                        lista = [0,x1,cargoAux,filename,"Estadual","ES"]
                        writer.writerow(lista)
                
                except Exception as e:
                    print(e)
                    continue

In [ ]:
# Merge Vila velha

f = open('C:\\temp\\vilavelha_leg\\CONSOLIDADO.csv', 'a')
for file in os.listdir("C:\\temp\\vilavelha_leg"):
    if file.endswith(".csv") and not file.startswith("CONSOLIDADO"):
        with open("C:\\temp\\vilavelha_leg\\"+file, 'r') as read_obj:
            for line in read_obj:
                f.write(line)
f.close()